In [1]:
# Librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [11]:
# Parametros de seguridad

MAX_I_R_PLE1 = 17
MAX_I_S_PLE1 = 18
MAX_I_T_PLE1 = 17

MAX_I_R_PLE7 = 13
MAX_I_S_PLE7 = 28
MAX_I_T_PLE7 = 32

TIEMPO_MIN = pd.Timedelta(minutes=5)


In [12]:
# Diccionario de parametros de seguridad

LIMITES_CORRIENTE = {
    'PLE1': {
        'R': MAX_I_R_PLE1,
        'S': MAX_I_S_PLE1,
        'T': MAX_I_T_PLE1,
    },
    'PLE7': {
        'R': MAX_I_R_PLE7,
        'S': MAX_I_S_PLE7,
        'T': MAX_I_T_PLE7,
    }
}

TIEMPO_MIN = pd.Timedelta(minutes=5)

In [13]:
# Abrir los graficos en el navegador

pio.renderers.default = 'browser'

In [14]:
# Funcion para cargar datos

from glob import glob
import pandas as pd

def cargar_maquina(base_path, maquina):
    paths = glob(f"{base_path}/{maquina}/**/*.csv", recursive=True)

    dfs = []
    for path in paths:
        df = pd.read_csv(path)
        df['maquina'] = maquina
        dfs.append(df)

    return (
        pd.concat(dfs, ignore_index=True)
          .sort_values('temporal_placa')
          .reset_index(drop=True)
    )


In [15]:
# Carga de datos

df_ple1 = cargar_maquina("../data", "PLE1")
df_ple7 = cargar_maquina("../data", "PLE7")

In [16]:
def preparar_df(df):
    df = df.copy()

    # Timestamp
    df['temporal_placa'] = pd.to_datetime(df['temporal_placa'])
    df['hora'] = df['temporal_placa'].dt.hour
    df['minuto'] = df['temporal_placa'].dt.minute

    # Turnos
    def asignar_turno(hora, minuto):
        t = hora * 60 + minuto

        # Pausas
        if 12*60 <= t < 12*60 + 30:
            return 'ALMUERZO'
        if 22*60 <= t < 22*60 + 30:
            return 'CENA'

        # Turnos
        if 5*60 <= t < 17*60:
            return 'TURNO MAÑANA'
        if 17*60 <= t < 22*60:
            return 'TURNO TARDE'
        if (t >= 22*60 + 30) or (t < 1*60):
            return 'TURNO TARDE'

        return 'FUERA_TURNO'

    df['turno'] = df.apply(
        lambda x: asignar_turno(x['hora'], x['minuto']),
        axis=1
    )
        
    # Potencias totales por timestamp
    df['p_activa_total'] = (
        df['potencia_a_r'] +
        df['potencia_a_s'] +
        df['potencia_a_t']
    )

    df['q_reactiva_total'] = (
        df['potencia_r_r'] +
        df['potencia_r_s'] +
        df['potencia_r_t']
    )

    return df


In [17]:

# Ajuste de datos de las plegadoras

df_ple1 = preparar_df(df_ple1)
df_ple7 = preparar_df(df_ple7)

df_all = pd.concat([df_ple1, df_ple7], ignore_index=True)

In [44]:
# Calculo la media de las corrientes excluyendo los picos

def media_sin_picos(s, q = 0.9):
    return s[s <= s.quantile(q)].mean()

medias_por_fase = (
    df_all
    .groupby('maquina')
    .agg(
        I_R_media=('corriente_r', media_sin_picos),
        I_S_media=('corriente_s', media_sin_picos),
        I_T_media=('corriente_t', media_sin_picos),
    )
)

medias_por_fase

,I_R_media,I_S_media,I_T_media
maquina,,,
PLE1,16.187595,17.210818,16.276914
PLE7,11.065071,23.275452,26.435411


In [18]:
# Alarma porsobrecorriente sostenida por fase

alarmas = []

for maq, g in df_all.groupby('maquina'):

    g = g.sort_values('temporal_placa').copy()

    if maq not in LIMITES_CORRIENTE:
        continue  # máquina sin límites definidos

    for fase, col in {'R': 'corriente_r',
                      'S': 'corriente_s',
                      'T': 'corriente_t'}.items():

        limite = LIMITES_CORRIENTE[maq][fase]

        # condición de sobrecorriente
        g['exceso'] = g[col] > limite

        # detectar bloques consecutivos
        g['bloque'] = (g['exceso'] != g['exceso'].shift()).cumsum()

        for _, b in g[g['exceso']].groupby('bloque'):

            duracion = b['temporal_placa'].iloc[-1] - b['temporal_placa'].iloc[0]

            if duracion >= TIEMPO_MIN:
                alarmas.append({
                    'maquina': maq,
                    'descripcion': 'Sobrecorriente sostenida por encima del límite',
                    'fase': fase,
                    'valor_medido': b[col].mean(),
                    'valor_limite': limite,
                    'exceso_pct': (b[col].mean() / limite - 1) * 100,
                    'fecha_primera_deteccion': b['temporal_placa'].iloc[0],
                    'duracion_min': duracion.total_seconds() / 60
                })

df_alarmas = pd.DataFrame(alarmas)
df_alarmas


,maquina,descripcion,fase,valor_medido,valor_limite,exceso_pct,fecha_primera_deteccion,duracion_min
0,PLE1,Sobrecorriente sostenida por encima del límite,R,17.316908,17,1.864166,2026-01-10 00:13:59-03:00,11.483333
1,PLE1,Sobrecorriente sostenida por encima del límite,S,18.660106,18,3.667255,2026-01-09 18:19:13-03:00,19.600000
2,PLE1,Sobrecorriente sostenida por encima del límite,S,19.094125,18,6.078471,2026-01-09 18:40:51-03:00,15.983333
3,PLE1,Sobrecorriente sostenida por encima del límite,S,18.423510,18,2.352835,2026-01-09 23:58:55-03:00,26.550000
4,PLE1,Sobrecorriente sostenida por encima del límite,T,18.771630,17,10.421354,2025-12-30 00:00:14-03:00,53.300000
5,PLE1,Sobrecorriente sostenida por encima del límite,T,17.262980,17,1.546944,2025-12-30 05:08:32-03:00,5.500000
6,PLE1,Sobrecorriente sostenida por encima del límite,T,17.146687,17,0.862864,2025-12-30 05:53:03-03:00,5.983333
7,PLE1,Sobrecorriente sostenida por encima del límite,T,17.354562,17,2.085659,2025-12-30 11:59:31-03:00,22.016667
8,PLE1,Sobrecorriente sostenida por encima del límite,T,17.235594,17,1.385849,2025-12-30 22:14:32-03:00,10.483333
9,PLE1,Sobrecorriente sostenida por encima del límite,T,17.730256,17,4.295622,2026-01-09 18:28:43-03:00,10.100000
